## Registration (non-rigid motion correction) of tiff movies using suite2p 
### options:<br>- make downsampled raw movie<br>- estimate lines offset through time (to be added)<br>- make downsamlped registered movie (for dislay and verification)
### this notebook runs on a suite2p environment

### imports

In [1]:
import time
import numpy as np
import tqdm
import suite2p

from mesoFlow.IO import *
from mesoFlow.movie import *
from mesoFlow.mesoFlow import *

### select raw data

In [2]:
### set path
# directory_path="/media/cstefens/data/data/test2_small" #linux box
directory_path="C:/Users/cstefens/OneDrive - Imperial College London/data/mouse1_lateral_path2_128x128" #desktop
directory_path="C:/Users/cstefens/OneDrive - Imperial College London/data/mouse1_lateral_path2_256x512" #desktop
getFilesPath(directory_path)

['C:/Users/cstefens/OneDrive - Imperial College London/data/mouse1_lateral_path2_256x512/movie-1.tif',
 'C:/Users/cstefens/OneDrive - Imperial College London/data/mouse1_lateral_path2_256x512/movie-2.tif']

In [3]:
### list files in a directory
os.listdir(directory_path)

['movie-1.tif', 'movie-2.tif']

### set parameters and options

In [4]:
# load default params
ops = suite2p.default_ops() 
ops["rescale_reg_tif"]=1.0,1.0,1.0 #default value
#ops #print default parameters

In [5]:
# modify default ops: set for registration only and save downsampled tiff
 
#Now, the motion corrected movie is saved as .bin and .tiff (option to resize tiff)
# TODO option to keep registered movie as bin file and/or tiff

ops["fs"]=7.66 #use getFpsScanImage(fname) on tif from scanImage
ops["tau"]=1.25 # timescale of the sensor (in seconds)
                #     0.7 for GCaMP6f
                #     1.0 for GCaMP6m
                #     1.25-1.5 for GCaMP6s
ops["roidetect"]=False # False for registration only
ops["do_bidiphase"]=True # compute bidirectional phase offset from misaligned line scanning 
ops["reg_tif"]=True #save registered tiffs
ops["rescale_reg_tif"]=0.5,0.5,0.1 #fx,fy,fz ratio for resizing registered tiff, for inspection/display purpose

### pipeline

In [6]:
print(">>>>>> "+directory_path)
t0_tot=time.time()

# option: save downsampled raw tiff
t0=time.time()
if not ops["rescale_reg_tif"] == (1.0, 1.0, 1.0):
    fxy=ops["rescale_reg_tif"][0]
    fz=ops["rescale_reg_tif"][2]
    generateDownsampledRawMovie([directory_path],fxy,fz)
print("\n>>>save downsampled raw tiff ",np.round((time.time()-t0)/60.0,2), " minutes\n")

# # TODO option: measure lines offset across time and apply correction
# t0=time.time()
# print("\n>>>measure line offset across time ",np.round((time.time()-t0)/60.0,2), " minutes\n")

# run suite2p registration
t0=time.time()
ops_reg=run_suite2p(ops,directory_path)
print("\n>>>suite2p registration ",np.round((time.time()-t0)/60.0,2), " minutes\n")

# option: downsampled registered tiff
t0=time.time()
if not ops["rescale_reg_tif"] == (1.0, 1.0, 1.0):
    downsampleTiffs(ops_reg["save_path"]+"/reg_tif",ops["rescale_reg_tif"])
    fname=concatenateTiffs_suite2p(ops_reg["save_path"]+"/reg_tif")
print("\n>>>downsampled registered tiff ",np.round((time.time()-t0)/60.0,2), " minutes\n")

print("\ntotal time ",np.round((time.time()-t0_tot)/60.0,2), " minutes\n")

>>>>>> C:/Users/cstefens/OneDrive - Imperial College London/data/mouse1_lateral_path2_256x512
C:/Users/cstefens/OneDrive - Imperial College London/data/mouse1_lateral_path2_256x512
1  /  2
2  /  2
(2170, 256, 128) uint16
200 frames saved
400 frames saved
600 frames saved
800 frames saved
1000 frames saved
1200 frames saved
1400 frames saved
1600 frames saved
1800 frames saved
2000 frames saved
(2170, 256, 128) uint16

>>>save downsampled raw tiff  0.1  minutes

{}
tif
** Found 2 tifs - converting to binary **
2000 frames of binary, time 5.62 sec.
4000 frames of binary, time 8.64 sec.
6000 frames of binary, time 12.49 sec.
8000 frames of binary, time 15.85 sec.
10000 frames of binary, time 19.48 sec.
12000 frames of binary, time 26.17 sec.
14000 frames of binary, time 29.99 sec.
16000 frames of binary, time 35.05 sec.
18000 frames of binary, time 39.64 sec.
20000 frames of binary, time 44.08 sec.
time 47.58 sec. Wrote 21700 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<

100%|██████████| 44/44 [00:08<00:00,  5.43it/s]


200 frames saved
400 frames saved
600 frames saved
800 frames saved
1000 frames saved
1200 frames saved
1400 frames saved
1600 frames saved
1800 frames saved
2000 frames saved

>>>downsampled registered tiff  0.15  minutes


total time  8.25  minutes

